In [88]:
!pip install pyspark

In [89]:
import pyspark
import pandas as pd
from pyspark.sql.types import *

In [90]:
import numpy as np
from pyspark.sql import SparkSession


In [91]:
import sys
import math

In [92]:
spark=SparkSession.builder.appName('firstapp').getOrCreate()

In [93]:
#load the dataset and add an index for each rows
data = spark.read.csv('./test.csv', header=True, inferSchema=True)
data = data.drop('Name')
df = data.toPandas()
df_index = pd.DataFrame({'index': range(0, len(df))})
result_df = pd.concat([df, df_index], axis=1)
data = spark.createDataFrame(result_df)
# asf = spark.createDataFrame(ee)
# asf.show()

In [94]:
#Select randomly five initials centroids
centers = data.toPandas().sample(n=5)
centers
# npCenters = spark.createDataFrame(interDf)
# centers = npCenters

,Age,X,Y,Z,index
20,525,300,42,123,20
19,500,285,40,117,19
23,600,345,48,141,23
8,225,120,18,51,8
9,250,135,20,57,9


In [121]:
#Map operation
cols = [item for item in data.columns if(item != 'index')]
def euclideanDistance(x, y):
    dist = 0
    for dim in cols:
        dist = dist + (x[dim] - y[dim])**2
    dist = math.sqrt(dist)
    return dist
def map(x, cent):
    minDist = sys.float_info.max
    index = -1
    for i, center in cent.iterrows():
        distCenter = euclideanDistance(x, center)
        if(minDist > distCenter):
             minDist = distCenter
             index = i
        y = x.asDict()
        # del y['index']
        y['num'] = 1
        toSave = Row(**y)         
    return (index, toSave)
# rdd = data.rdd.map(lambda x: map(x)).groupByKey()
# e = rdd.toDF()
# for f in e.collect():
#     print(f)


In [122]:
def reduce(a, b):
    c = {}
    for column in cols:
        c[column] = float(a[column]) + float(b[column])
    c['num'] = float(a['num']) + float(b['num'])
    toSave = Row(**c)
    return toSave
def computeMean(y):
    g = y[1].asDict()
    for col in cols:
        g[col] = g[col]/g['num']
    g['index'] = y[0]
    return Row(**g)

In [ ]:
def objFunction(x,cent):
    result = 0
    for i, center in cent.iterrows():
        result = euclideanDistance(x, center) + result

In [124]:
centers = data.toPandas().sample(n=5)
for i in range(0, 5):
    mapResult = data.rdd.map(lambda x: map(x, centers))
    
    reduceResult = mapResult.reduceByKey(lambda a,b: reduce(a, b)).map(lambda q: computeMean(q))
    e = reduceResult.toDF()
    for f in e.collect():
        print(f)
    print()
    print()
    centers = e.toPandas()
    error = data.rdd.map(lambda x: euclideanDistance())

Row(Age=50.0, X=16.666666666666668, Y=4.0, Z=9.0, num=3.0, index=1)
Row(Age=100.0, X=45.0, Y=8.0, Z=21.0, num=None, index=1)
Row(Age=125.0, X=60.0, Y=10.0, Z=27.0, num=None, index=1)
Row(Age=200.0, X=105.0, Y=16.0, Z=45.0, num=5.0, index=6)
Row(Age=500.0, X=285.0, Y=40.0, Z=117.0, num=19.0, index=13)




Row(Age=50.0, X=16.666666666666668, Y=4.0, Z=9.0, num=3.0, index=0)
Row(Age=100.0, X=45.0, Y=8.0, Z=21.0, num=None, index=1)
Row(Age=137.5, X=67.5, Y=11.0, Z=30.0, num=2.0, index=2)
Row(Age=262.5, X=142.5, Y=21.0, Z=60.0, num=8.0, index=3)
Row(Age=550.0, X=315.0, Y=44.0, Z=129.0, num=15.0, index=4)


Row(Age=50.0, X=16.666666666666668, Y=4.0, Z=9.0, num=3.0, index=0)
Row(Age=100.0, X=45.0, Y=8.0, Z=21.0, num=None, index=1)
Row(Age=162.5, X=82.5, Y=13.0, Z=36.0, num=4.0, index=2)
Row(Age=312.5, X=172.5, Y=25.0, Z=72.0, num=8.0, index=3)
Row(Age=575.0, X=330.0, Y=46.0, Z=135.0, num=13.0, index=4)


Row(Age=50.0, X=16.666666666666668, Y=4.0, Z=9.0, num=3.0, index=0)
Row(Age=112.5, X=52.5, Y=9.0, Z=24.0, num=2.0, index=1)
Row(Age=187.5, X=97.5, Y=15.0, Z=42.0, num=4.0, index=2)
Row(Age=337.5, X=187.5, Y=27.0, Z=78.0, num=8.0, index=3)
Row(Age=587.5, X=337.5, Y=47.0, Z=138.0, num=12.0, index=4)


Row(Age=50.0, X=16.666666666666668, Y=4.0, Z=9.0, num=3.0, index=0)
Row(Age=125.0, X=60.0, Y=10